In [1]:
import numpy as np
import pandas as pd
import re
from os.path import join

from tools import *
from S2.utils._path import *

from sklearn.decomposition import LatentDirichletAllocation
from scipy.spatial import distance

import matplotlib.pyplot as plt
import numpy as np

## rename EXTRACT_FEATURES to DIMENSION_REDUCTION
read_prefix = "/data/MIMIC3/"
write_prefix = "/data/liu/LDA"
res_prefix = "/data/liu/LDA/lda_result"
res_r_prefix = "/data/liu/LDA/lda_R_result/"

## Create Feature representatives combien five tables

In [2]:
sample_epis_file='HADM_ID_SAMPLE_PER_PATIENT'
sampled_hadmid = read_data(join(singledrug_prefix,sample_epis_file),dtype=str)
sampled_hadmid.head()

,SUBJECT_ID,HADM_ID
0,2,163353
1,4,185777
2,6,107064
3,8,159514
4,9,150750


In [3]:
def print_statistics(df):
    
    print("# of rows: %d"%len(df))
    print("# of patients: %d"%len(df['SUBJECT_ID'].unique()))
    print("# of episodes: %d"%len(df['HADM_ID'].unique()))

In [4]:
def df2matrix(df):
    cols=["",df.columns[-1]]
    print("Original Data:")
    print_statistics(df)
    df_sampled = left_join(sampled_hadmid,df,list(sampled_hadmid.columns))
    print("Sampled Data:")
    print_statistics(df_sampled)
    
    
    
    # pres_matrix = pres_df_1k_1k_hadm.pivot(index='SUBJECT_ID', columns='NDC', values='VALUE').fillna(0).reset_index()

    return df_sampled

In [21]:
from IPython.display import HTML
HTML(filename='/data/risa/annotate/annotate_9981_125955_1.html')

In [19]:
## ICD9_CODE, 1
diag_df = read_data(join(read_prefix,"DIAGNOSES_ICD"),dtype={"SUBJECT_ID":str,"HADM_ID":str})
diag_df.head()
df2matrix(diag_df,"aa")

Original Data:/n
# of rows: 651047
# of patients: 46520
# of patients: 58976
Sampled Data:/n
# of rows: 445514
# of patients: 38763
# of patients: 38763


,SUBJECT_ID,HADM_ID,ROW_ID,SEQ_NUM,ICD9_CODE
0,2,163353,1,1.0,V3001
1,2,163353,2,2.0,V053
2,2,163353,3,3.0,V290
3,4,185777,13,1.0,042
4,4,185777,14,2.0,1363
...,...,...,...,...,...
445509,99999,113369,651043,1.0,75612
445510,99999,113369,651044,2.0,7861
445511,99999,113369,651045,3.0,4019
445512,99999,113369,651046,4.0,25000


In [12]:
pres_df = read_data(
    join(read_prefix,'PRESCRIPTIONS'),
    dtype={"SUBJECT_ID":str,"HADM_ID":str,"NDC":str})[["SUBJECT_ID","HADM_ID","NDC"]].dropna(subset=['NDC'])
pres_df = pres_df[pres_df['NDC']!=0]
# read_data(join(read_prefix,"DIAGNOSES_ICD"),dtype={"SUBJECT_ID":str,"HADM_ID":str})

df2matrix(pres_df,"aa")

Original Data:
# of rows: 4151987
# of patients: 39360
# of patients: 50212
Sampled Data:
# of rows: 3167448
# of patients: 38763
# of patients: 38763


,SUBJECT_ID,HADM_ID,NDC
0,2,163353,63323017302
1,2,163353,0
2,2,163353,63323038810
3,2,163353,0
4,4,185777,0
...,...,...,...
3167443,99999,113369,66993084225
3167444,99999,113369,66993084225
3167445,99999,113369,66993084225
3167446,99999,113369,66993084225


In [5]:
## ITEMID, VALUE
lab_df=read_data(
    join(read_prefix,'LABEVENTS'),
    dtype={"SUBJECT_ID":str,"HADM_ID":str})[["SUBJECT_ID","HADM_ID","ITEMID","VALUE"]].dropna(subset=['VALUE'])
df2matrix(lab_df,"aa")

Original Data:
# of rows: 27852810
# of patients: 46252
# of patients: 58152
Sampled Data:
# of rows: 15412168
# of patients: 38763
# of patients: 38763


,SUBJECT_ID,HADM_ID,ITEMID,VALUE
0,2,163353,51143.0,0
1,2,163353,51144.0,0
2,2,163353,51146.0,0
3,2,163353,51200.0,0
4,2,163353,51221.0,0
...,...,...,...,...
15412163,99999,113369,51250.0,93
15412164,99999,113369,51265.0,193
15412165,99999,113369,51277.0,12.9
15412166,99999,113369,51279.0,3.03


In [6]:
temp_df = df2matrix(lab_df,"")
# [["HADM_ID","ITEMID","VALUE"]].drop_duplicates().pivot(index='HADM_ID', columns='ITEMID', values='VALUE').reset_index()
# matrix.head()


Original Data:
# of rows: 27852810
# of patients: 46252
# of patients: 58152
Sampled Data:
# of rows: 15412168
# of patients: 38763
# of patients: 38763


In [9]:
temp_df.groupby(["HADM_ID","ITEMID"])["VALUE"].mean()

DataError: No numeric types to aggregate

In [14]:
temp_df["real"] = temp_df["VALUE"].apply(lambda s: pd.to_numeric(s, errors='coerce'))
temp_df

,SUBJECT_ID,HADM_ID,ITEMID,VALUE,real
0,2,163353,51143.0,0,0.00
1,2,163353,51144.0,0,0.00
2,2,163353,51146.0,0,0.00
3,2,163353,51200.0,0,0.00
4,2,163353,51221.0,0,0.00
...,...,...,...,...,...
15412163,99999,113369,51250.0,93,93.00
15412164,99999,113369,51265.0,193,193.00
15412165,99999,113369,51277.0,12.9,12.90
15412166,99999,113369,51279.0,3.03,3.03


In [16]:
print(len(temp_df))


15412168


NameError: name 'lenn' is not defined

In [19]:
from S2.cluster_analysis import feature_creation

In [17]:
print(len(temp_df.drop_duplicates()))

10417738


In [ ]:
fc = feature_creation()
tt1 = fc.sampling(temp_df,1,["HADM_ID","ITEMID"],"VALUE")

In [15]:
temp_df[temp_df["real"].isna()]

,SUBJECT_ID,HADM_ID,ITEMID,VALUE,real
18,2,163353,51137.0,1+,NaN
25,2,163353,51233.0,NORMAL,NaN
27,2,163353,51246.0,1+,NaN
32,2,163353,51252.0,NORMAL,NaN
38,2,163353,51266.0,NORMAL,NaN
...,...,...,...,...,...
15411818,99992,197084,51506.0,Clear,NaN
15411819,99992,197084,51508.0,Yellow,NaN
15411820,99992,197084,51514.0,NEG,NaN
15411973,99995,137810,51519.0,NONE,NaN


In [9]:
## procedures
procedure_df=read_data(
    join(read_prefix,'PROCEDURES_ICD'),
    dtype={"SUBJECT_ID":str,"HADM_ID":str})
procedure_df.head()


,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [10]:
df2matrix(procedure_df,"aa")

Original Data:
# of rows: 240095
# of patients: 42214
# of patients: 52243
Sampled Data:
# of rows: 174280
# of patients: 38763
# of patients: 38763


,SUBJECT_ID,HADM_ID,ROW_ID,SEQ_NUM,ICD9_CODE
0,2,163353,109376.0,1.0,9955.0
1,4,185777,74266.0,1.0,3893.0
2,4,185777,74267.0,2.0,8872.0
3,4,185777,74268.0,3.0,3323.0
4,6,107064,91635.0,1.0,5569.0
...,...,...,...,...,...
174275,99999,113369,174285.0,1.0,8108.0
174276,99999,113369,174286.0,2.0,8051.0
174277,99999,113369,174287.0,3.0,8162.0
174278,99999,113369,174288.0,4.0,9979.0


## Check endline of extracted sections

In [2]:
## func list delta
import io

def list_delta(str1, str2):
    l1=[i for i in str1.split(",") if i!="None"]
    l2=[i for i in str2.split(",") if i!="None"]
    if(l1):
        delta = [y-x for x,y in zip(map(int,l1),map(int,l2))]
        return delta
    else:
        return None


In [3]:
dissum_df = read_data("/data/liu/mimic3/CLAMP_NER/input/ReRule2_Discharge summary_Only",dtype={"HADM_ID":str})
dissum_df.head()

,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT,TITLE,TTITLE_LINE,SECTION_END_LINE,NEXT_TITLE_LINE,NEXT_TITLE,HOSPITAL COURSE,MEDICAL HISTORY,MEDICATIONS ON ADMISSION
0,11,194540,1,Admission Date: [**2178-4-16**] ...,"brief hospital course:,past medical history:,d...","253,34,281","276,37,318","278,39,320","medications on admission:,family history:,disc...",\nA/P: 50 year-old woman without significant P...,"\nNewly diagnosed GBM as above\notherwise, none\n",\n1. Diabetic.com Starter Kit Kit Sig: One...
1,17,161087,1,Admission Date: [**2135-5-9**] D...,"brief hospital course:,past medical history:,d...","57,24,74","69,29,99","70,30,101","medications on admission:,social history:,disc...",\nAdmitted through ER as above and referred to...,\ns/p min. inv. closure of Patent foramen oval...,\n1. Docusate Sodium 100 mg Capsule Sig: One (...
2,17,194023,1,Admission Date: [**2134-12-27**] ...,"brief hospital course:,past medical history:,d...","63,27,87","82,31,100","83,32,102","medications on admission:,social history:,disc...",\nMrs. [**Known lastname 11679**] was admitted...,\nPatent foramen ovale; History of Stroke/TIA;...,\n1. Bupropion 150 mg Tablet Sustained Release...
3,18,188822,1,Admission Date: [**2167-9-30**] ...,"brief hospital course:,past medical history:","124,38,None","142,44,None","143,45,None","therapeutic interventions:,past psychiatric hi...",\nSOMATIC INTERVENTIONS:\nThe patient was cont...,"\n- IDDM, Type II\n- OSA\n- Borderline HTN\n- ...",NaN
4,18,188822,2,Admission Date: [**2167-10-2**] ...,"brief hospital course:,past medical history:,d...","95,31,168","156,37,204","157,38,206","medications on admission:,past psychiatric his...","\nA/P: 50M with history of hypertension, type ...","\n- IDDM, Type II\n- OSA\n- Borderline HTN\n- ...",\n1. Nortriptyline 50 mg Capsule Sig: One (1) ...


In [4]:
def get_one_row(df, row_id):
    
    for col, value in zip(df.columns[4:9],df.loc[row_id,df.columns[4:9]]):
        print('{} :\n {}\n'.format(col,value))

    print("\n{}TEXT{}\n".format("-"*45,"-"*45))

    with io.StringIO(df.loc[row_id,"TEXT"]) as f:
        for i, line in enumerate(f, start=1):
            print('{} : {}'.format(i, line.strip()))
            
    print("\n{}MEDICATIONS ON ADMISSION{}\n".format("-"*45,"-"*45))
    print(df.loc[row_id,"MEDICATIONS ON ADMISSION"])

In [9]:
## create benchamrk
dissum_df["delta"]=dissum_df[["SECTION_END_LINE","NEXT_TITLE_LINE"]].apply(lambda x: list_delta(*x), axis=1)
dissum_df["check delta in [1,2]"]=dissum_df["delta"].apply(lambda x: all(delta in [1,2] for delta in x))

In [7]:
dissum_df.head()

,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT,TITLE,TTITLE_LINE,SECTION_END_LINE,NEXT_TITLE_LINE,NEXT_TITLE,HOSPITAL COURSE,MEDICAL HISTORY,MEDICATIONS ON ADMISSION,delta,"check delta in [1,2]"
0,11,194540,1,Admission Date: [**2178-4-16**] ...,"brief hospital course:,past medical history:,d...","254,35,282","277,38,319","279,40,321","medications on admission:,family history:,disc...",\nA/P: 50 year-old woman without significant P...,"\nNewly diagnosed GBM as above\notherwise, none\n",\n1. Diabetic.com Starter Kit Kit Sig: One...,"[2, 2, 2]",True
1,17,161087,1,Admission Date: [**2135-5-9**] D...,"brief hospital course:,past medical history:,d...","58,25,75","70,30,100","71,31,102","medications on admission:,social history:,disc...",\nAdmitted through ER as above and referred to...,\ns/p min. inv. closure of Patent foramen oval...,\n1. Docusate Sodium 100 mg Capsule Sig: One (...,"[1, 1, 2]",True
2,17,194023,1,Admission Date: [**2134-12-27**] ...,"brief hospital course:,past medical history:,d...","64,28,88","83,32,101","84,33,103","medications on admission:,social history:,disc...",\nMrs. [**Known lastname 11679**] was admitted...,\nPatent foramen ovale; History of Stroke/TIA;...,\n1. Bupropion 150 mg Tablet Sustained Release...,"[1, 1, 2]",True
3,18,188822,1,Admission Date: [**2167-9-30**] ...,"brief hospital course:,past medical history:","125,39,None","143,45,None","144,46,None","therapeutic interventions:,past psychiatric hi...",\nSOMATIC INTERVENTIONS:\nThe patient was cont...,"\n- IDDM, Type II\n- OSA\n- Borderline HTN\n- ...",NaN,"[1, 1]",True
4,18,188822,2,Admission Date: [**2167-10-2**] ...,"brief hospital course:,past medical history:,d...","96,32,169","157,38,205","158,39,207","medications on admission:,past psychiatric his...","\nA/P: 50M with history of hypertension, type ...","\n- IDDM, Type II\n- OSA\n- Borderline HTN\n- ...",\n1. Nortriptyline 50 mg Capsule Sig: One (1) ...,"[1, 1, 2]",True


In [6]:
# get_one_row(dissum_df,14)

In [5]:
# get_one_row(dissum_df,49)

In [4]:
# get_one_row(dissum_df,37399)

In [3]:
# all(word in "ADMISSION MEDICATION" for word in ["MEDICATIONS","ADMISSION"])

In [2]:
# dissum_df.loc[dissum_df["check delta in [1,2]"]]

In [1]:
# dissum_df.loc[~dissum_df["check delta in [1,2]"]]

## Debug
line number

In [5]:
dissum_df[dissum_df["HADM_ID"]=="137810"]

,SUBJECT_ID,HADM_ID,GROUP_RANK,TEXT,TITLE,TTITLE_LINE,SECTION_END_LINE,NEXT_TITLE_LINE,NEXT_TITLE,HOSPITAL COURSE,MEDICAL HISTORY,MEDICATIONS ON ADMISSION
37399,99995,137810,1,Admission Date: [**2147-2-8**] D...,"brief hospital course:,past medical history:,m...","73,32,133","82,44,141","83,46,142","medications on admission:,social history:,what...",\nThe patient was brought to the operating roo...,\nPAST MEDICAL/SURGICAL HISTORY:\nCAD\nDiabete...,\n??????\tTake Aspirin 325mg (enteric coated) ...


In [5]:
from S2.dis_sum_extraction import *
test_text=dissum_df[dissum_df["HADM_ID"]=="137810"]['TEXT'][37399]
# re_split(dissum_df[dissum_df["HADM_ID"]=="137810"]['TEXT'][37399])

In [12]:
# dissum_df[dissum_df["HADM_ID"]=="137810"]['TEXT'][37399]